In [1]:
import pyarrow
import pandas as pd

from bs4 import BeautifulSoup

#для очистки текста
import re

In [2]:
data_test = pd.read_parquet("test.parquet")

In [3]:
data_train = pd.read_parquet("train.parquet")

In [4]:
data_train.head()

product_id  category_id   sale  shop_id           shop_title  rating  \
0      325286        12171  False     9031                Aksik     5.0   
1      888134        14233  False    18305                 Sela     5.0   
3     1267173        13429  False    16357  ЮНЛАНДИЯ канцтовары     5.0   
4     1416943         2789  False    34666        вася-nicotine     4.0   
5     1058275        12834  False    26389           Lim Market     4.6   

                                         text_fields  \
0  {"title": "Зарядный кабель Borofone BX1 Lightn...   
1  {"title": "Трусы Sela", "description": "Трусы-...   
3  {"title": "Гуашь \"ЮНЫЙ ВОЛШЕБНИК\", 12 цветов...   
4  {"title": "Колба для кальяна Крафт (разные цве...   
5  {"title": "Пижама женская, однотонная с шортам...   

                                       category_name  
0  Все категории->Электроника->Смартфоны и телефо...  
1  Все категории->Одежда->Женская одежда->Белье и...  
3  Все категории->Хобби и творчество->Рисование->...  
4  Все категории->Хобби и творчество->Товары для ...  
5  Все категории->Одежда->Женская одежда->Домашня...

category_id - категория товара (таргет). Целевая метка!
product_id - идентификатор товара


Хотим научиться предсказывать категорию на основе описания, картинки и других параметрах товаров.
Ниже приведем столбцы, которые нам НЕ НУЖНЫ для решения задачи:
sale - флаг, обозначающий находится ли товар в распродаже;
shop_id (seller_id) - id магазина (id-продавца);
shop_title - название магазина;
rating - рейтинг магазина

Чтобы составить входной столбце для обучения на текстовом описании предлагаю взять:
1) последнее слово в category_name (после последней ->). Несёт больше всего информации для предсказания category_id

2) title из text_fields. Название товара, тоже несёт много информации для category_id

3) description из text_fields

In [5]:
#Для работы с изображениями нам понадобятся product_id (название файла .img) и category_id (целевая метка)
data_train[['product_id', 'category_id']].to_csv('train_images.csv', sep='\t')
data_test[['product_id']].to_csv('test_images.csv', sep='\t')

#train_images = pd.read_csv('train_images.csv', sep='\t', index_col=0)
#test_images = pd.read_csv('test_images.csv', sep='\t', index_col=0)

In [6]:
def text_converter(text):
    #clean emojis
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    text_without_emoji = emoji_pattern.sub(r'', text)
    
    #clean punctuation marks
    text_final = re.sub(r'[^\w]', ' ', text_without_emoji)
    
    return str.lower(text_final)

In [7]:
def cat_name_add(df):
    df_new = df.copy()
    cat_names_lst = []
    for i in range(len(df_new)):
        cat_name_full = df['category_name'].iloc[i]
        cat_name_list = cat_name_full.split("->")
        cat_name_current = cat_name_list[-1]
        
        cat_names_lst.append(text_converter(cat_name_current))
        
    df_new['cat_name'] = cat_names_lst
    return df_new

In [8]:
def titles_add(df):
    df_new = df.copy()
    titles_lst = []
    
    dict_text_fields_all = list(df_new['text_fields'])
    
    for dict_text_fields_current in dict_text_fields_all:
        #get Title
        dict_text_fields = eval(dict_text_fields_current)
        titles_lst.append(text_converter(dict_text_fields['title']))
        
    df_new['titles'] = titles_lst
    return df_new

In [9]:
def description_add(df):
    df_new = df.copy()
    description_lst = []
    
    dict_text_fields_all = list(df_new['text_fields'])
    
    for dict_text_fields_current in dict_text_fields_all:
        
        #Get Description
        dict_text_fields = eval(dict_text_fields_current)
        parse_line = dict_text_fields['description']
        
        if (parse_line) != '':
            soup = BeautifulSoup(parse_line)
            p_parser = soup.body.find_all('p')
            
            text_str=''
            for p_current in p_parser:
                text_current = p_current.text
                text_str += str(text_current)
                
            text_converted = text_converter(text_str)[:50] #достаточно
            description_lst.append(text_converted)
            
        else:
            description_lst.append(' ')
        
    df_new['description'] = description_lst
    return df_new

In [10]:
def train_data_preparation(df):
    df_new = df.copy()
    df_new1 = cat_name_add(df)
    df_new2 = titles_add(df_new1)
    df_new3 = description_add(df_new2)
    
    df_new3 = df_new3.assign(Docs=[str(x) + ' ' + str(y) + ' ' + str(z) for x, y, z in zip(df_new3['cat_name'], df_new3['titles'], df_new3['description'])])
    df_new3.Docs = df_new3.Docs.astype('string')
    df_new3.drop(['cat_name', 'titles', 'description'], axis=1, inplace=True)
    
    df_final = df_new3.drop(['sale', 'shop_id', 'shop_title', 'rating',
                            'product_id', 'text_fields', 'category_name'], axis=1)
    
    return df_final

In [11]:
df_train = train_data_preparation(data_train)

In [12]:
df_train.head(10)

category_id                                               Docs
0         12171  кабели зарядный кабель borofone bx1 lightning ...
1         14233  трусы трусы sela трусы слипы из эластичного бе...
3         13429  краски  пигменты гуашь  юный волшебник   12 цв...
4          2789  колбы колба для кальяна крафт  разные цвета  у...
5         12834  пижамы пижама женская  однотонная с шортами лё...
6         13337  напальчники и накладки накладка чехол на стики...
7         12909  носки и подследники детские теплые носки детск...
8         11937  чехлы силиконовый чехол для айфон iphone 11  1...
9         11956  футболки футболка женская твое  повседневная  ...
10        14922  защитные стекла защитное стекло гибкое  пленка...

In [13]:
#Есть такие category_id, которые встречаются только один раз
# Вывод: ДАТАСЕТ ОЧЕНЬ НЕСБАЛАНСИРОВАНЫЙ
df_train['category_id'].value_counts()

11937    6590
14922    3709
13651    1463
13143    1460
12980    1222
         ... 
12808       2
12901       1
11549       1
11875       1
12836       1
Name: category_id, Length: 874, dtype: int64

In [14]:
df_train['category_id'].value_counts().describe()

count     874.000000
mean      104.256293
std       300.107191
min         1.000000
25%        14.000000
50%        38.000000
75%        99.000000
max      6590.000000
Name: category_id, dtype: float64

In [15]:
def test_data_preparation(df):
    df_new = df.copy()
    
    df_new2 = titles_add(df_new)
    df_new3 = description_add(df_new2)
    
    df_new3 = df_new3.assign(Docs=[str(x) + ' ' + str(y) for x, y in zip(df_new3['titles'], df_new3['description'])])
    df_new3.Docs = df_new3.Docs.astype('string')
    df_new3.drop(['titles', 'description'], axis=1, inplace=True)
    
    df_final = df_new3.drop(['sale', 'shop_id', 'shop_title', 
                             'rating', 'text_fields'], axis=1)
    
    return df_final

In [16]:
df_test = test_data_preparation(data_test)

In [17]:
df_test.head()

product_id                                               Docs
1     1997646  светодиодная лента smart led strip light  с пу...
2      927375  стекло пленка керамик матовое honor 50 lite 10...
3     1921513  проводные наушники с микрофоном jack 3 5  ios ...
4     1668662  декоративная табличка  правила кухни   подстав...
5     1467778  подставка под ложку керамическая  подложка  кл...

In [18]:
#Сохраним данные для обучения
df_train.to_parquet('train_data.parquet')
df_test.to_parquet('test_data.parquet')

На этих данных будем обучаться в папке "1 Fast Text Imbalanced"